In [3]:
import numpy as np

def als_tt(tensor, ranks, max_iter=100, tol=1e-6, random_seed=None):
    """
    Perform ALS to decompose an order-d tensor into Tensor Train (TT) format.

    Parameters:
    - tensor: numpy array, the input order-d tensor
    - ranks: list of int, the TT-ranks (length d-1)
    - max_iter: int, maximum number of iterations
    - tol: float, convergence tolerance
    - random_seed: int, random seed for reproducibility

    Returns:
    - cores: list of numpy arrays, the TT cores
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # Tensor dimensions and order
    dims = tensor.shape
    d = len(dims)  # Order of the tensor
    
    # Initialize TT cores randomly
    cores = []
    for k in range(d):
        if k == 0:
            shape = (1, dims[k], ranks[k])  # First core
        elif k == d - 1:
            shape = (ranks[k - 1], dims[k], 1)  # Last core
        else:
            shape = (ranks[k - 1], dims[k], ranks[k])  # Middle cores
        cores.append(np.random.randn(*shape))
    
    # ALS optimization
    for iteration in range(max_iter):
        for k in range(d):
            # Unfold the tensor and the cores
            tensor_unfolded = unfold(tensor, k)
            cores_unfolded = [unfold(core, 1) for core in cores]
            
            # Compute the left and right products
            left = np.ones((1, 1))  # Initialize left product
            for i in range(k):
                left = np.kron(left, cores_unfolded[i])
            
            right = np.ones((1, 1))  # Initialize right product
            for i in range(k + 1, d):
                right = np.kron(right, cores_unfolded[i])
            
            # Solve for the current core using least squares
            A = np.kron(left.T, right.T)
            b = tensor_unfolded.flatten()
            core_k_new = np.linalg.lstsq(A, b, rcond=None)[0]
            core_k_new = core_k_new.reshape(cores[k].shape)
            
            # Update the core
            cores[k] = core_k_new
        
        # Check for convergence
        if iteration > 0:
            error = np.linalg.norm(tensor - reconstruct_tt(cores))
            if error < tol:
                print(f"Converged at iteration {iteration}")
                break
    
    return cores

def unfold(tensor, mode):
    """
    Unfold a tensor along a given mode.
    """
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))

def reconstruct_tt(cores):
    """
    Reconstruct the tensor from its TT cores.
    """
    d = len(cores)
    tensor = cores[0]
    for k in range(1, d):
        tensor = np.tensordot(tensor, cores[k], axes=(-1, 0))
    return tensor

# Example usage
if __name__ == "__main__":
    # Example order-3 tensor
    tensor = np.random.randn(3, 4, 5)
    
    # TT ranks (must have length d-1)
    ranks = [2, 3]
    
    # Perform ALS for TT decomposition
    cores = als_tt(tensor, ranks, random_seed=42)
    
    # Reconstruct the tensor
    reconstructed_tensor = reconstruct_tt(cores)
    
    # Check the reconstruction error
    error = np.linalg.norm(tensor - reconstructed_tensor)
    print(f"Reconstruction Error: {error}")

LinAlgError: Incompatible dimensions